# 스타벅스 스크래핑

In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import pandas as pd

In [13]:
# webdriver로 browser 준비
browser = webdriver.Chrome()
browser.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

### CSS selector로 위치를 찾아서 클릭하기
 browser에서 모든 정보가 로딩된 뒤에 click해야 제대로 작동하기 때문에 sleep을 설정하는 것이 좋다. 그러나 Jupyter notebook에서는 cell이 하나씩 순차적으로 실행되니까 크게 상관이 없긴 하다. <br>
 또는 전체를 run하는 경우, 너무 빨리 지나가버리기 때문에 sleep을 설정하면 진행을 천천히 확인할 수 있다.

In [14]:
import time
time.sleep(10)

browser.find_element_by_css_selector('ul.sido_arae_box > li > a[data-sidocd="01"]').click()    # '서울' 클릭

In [15]:
time.sleep(10)
browser.find_element_by_css_selector('ul.gugun_arae_box > li > a[href*="0"][data-guguncd=""]').click()   # '전체' 클릭

위 CSS selector 중 `[href*="0"]` 부분은 selector를 특정하는데 도움이 되지 않아서 필요 없음. 단지 tag attribute를 여러개 연이어 사용할 수 있다는 것을 보여주기 위해서 추가함

### BeautifulSoup으로 매장 리스트를 읽어오기

In [16]:
page = browser.page_source            # 현재 webdriver에 열려있는 페이지의 내용을 불러옴
soup = BS(page, 'html.parser')

In [18]:
tags = soup.select('li.quickResultLstCon')
len(tags), tags[0]                        # 총 554개 매장

(554,
 <li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>)

In [19]:
# 매장이름, 주소, 위도, 경도, 매장타입을 스크래핑
starbucks = []

for tag in tags:
    name = tag['data-name']
    add = tag.select_one('p').text
    lat = tag['data-lat']
    long = tag['data-long']
    store = tag.select_one('i').text
    starbucks.append([name, add, lat, long, store])

starbucks[:5]

[['역삼아레나빌딩',
  '서울특별시 강남구 언주로 425 (역삼동)1522-3232',
  '37.501087',
  '127.043069',
  '리저브 매장 2번'],
 ['논현역사거리',
  '서울특별시 강남구 강남대로 538 (논현동)1522-3232',
  '37.510178',
  '127.022223',
  '리저브 매장 2번'],
 ['신사역성일빌딩',
  '서울특별시 강남구 강남대로 584 (논현동)1522-3232',
  '37.514132',
  '127.020563',
  '리저브 매장 2번'],
 ['국기원사거리',
  '서울특별시 강남구 테헤란로 125 (역삼동)1522-3232',
  '37.499517',
  '127.031495',
  '리저브 매장 2번'],
 ['스탈릿대치R',
  '서울특별시 강남구 남부순환로 2947 (대치동)1522-3232',
  '37.494668',
  '127.062583',
  '리저브 매장 2번']]

### DataFrame으로 만들기

In [20]:
pd.DataFrame(starbucks, columns=['Store', 'Address', 'Latitude', 'Longitude', 'Type']).head()

,Store,Address,Latitude,Longitude,Type
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동)1522-3232,37.501087,127.043069,리저브 매장 2번
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동)1522-3232,37.510178,127.022223,리저브 매장 2번
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동)1522-3232,37.514132,127.020563,리저브 매장 2번
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동)1522-3232,37.499517,127.031495,리저브 매장 2번
4,스탈릿대치R,서울특별시 강남구 남부순환로 2947 (대치동)1522-3232,37.494668,127.062583,리저브 매장 2번


---
### 번외) WebElement object의 html source 확인하는 법
Selenium으로 element 찾기를 마친 뒤 BeautifulSoup으로 parsing 할 수도 있다!

In [21]:
# outerHTML: 지정한 태그(li) 포함해서 가져옴
# innerHTML: 지정한 태그 제외 안쪽만 가져옴
elmt = browser.find_element_by_css_selector('li.quickResultLstCon').get_attribute('outerHTML')
elmt, type(elmt)                  # str

('<li class="quickResultLstCon" style="background:#fff" data-lat="37.501087" data-long="127.043069" data-index="0" data-name="역삼아레나빌딩" data-code="3762" data-storecd="1509" data-hlytag="null">\t<strong data-store="1509" data-yn="N" data-name="역삼아레나빌딩" data-my_siren_order_store_yn="N">역삼아레나빌딩  </strong>\t<p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br>1522-3232</p>\t<i class="pin_general">리저브 매장 2번</i></li>',
 str)

In [22]:
print(BS(elmt, 'html.parser').prettify())

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff">
 <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">
  역삼아레나빌딩
 </strong>
 <p class="result_details">
  서울특별시 강남구 언주로 425 (역삼동)
  <br/>
  1522-3232
 </p>
 <i class="pin_general">
  리저브 매장 2번
 </i>
</li>


### Selenium만 사용해서 Starbucks 매장정보 가져오기

In [23]:
# 매장정보 모두 가져오기
tags = browser.find_elements_by_css_selector('li.quickResultLstCon')
tags[0].get_attribute('outerHTML')          # 첫번째 WebElement의 html source 확인
# tags[0].find_element_by_css_selector('p').text   # html은 제대로 가져오는데 왜 text는 안 불러

'<li class="quickResultLstCon" style="background:#fff" data-lat="37.501087" data-long="127.043069" data-index="0" data-name="역삼아레나빌딩" data-code="3762" data-storecd="1509" data-hlytag="null">\t<strong data-store="1509" data-yn="N" data-name="역삼아레나빌딩" data-my_siren_order_store_yn="N">역삼아레나빌딩  </strong>\t<p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br>1522-3232</p>\t<i class="pin_general">리저브 매장 2번</i></li>'

In [24]:
# 속도가 너무 느리다...
stores = []

for tag in tags:
    name = tag.get_attribute('data-name')        # tag_name, css_selector로는 text가 안 불러와짐
    add = tag.find_element_by_xpath('//*[@id="mCSB_3_container"]/ul/li[5]/p').text.replace('\n', ' ') 
    lat = tag.get_attribute('data-lat')
    long = tag.get_attribute('data-long')
    store = tag.find_element_by_xpath('//*[@id="mCSB_3_container"]/ul/li[5]/i').text
    stores.append([name, add, lat, long, store])
    
stores[:5]

[['역삼아레나빌딩', '', '37.501087', '127.043069', ''],
 ['논현역사거리', '', '37.510178', '127.022223', ''],
 ['신사역성일빌딩', '', '37.514132', '127.020563', ''],
 ['국기원사거리', '', '37.499517', '127.031495', ''],
 ['스탈릿대치R', '', '37.494668', '127.062583', '']]

In [25]:
# Dataframe으로 만들기
pd.DataFrame(stores, columns=['Store', 'Address', 'Latitude', 'Longitude', 'Type']).head()

,Store,Address,Latitude,Longitude,Type
0,역삼아레나빌딩,,37.501087,127.043069,
1,논현역사거리,,37.510178,127.022223,
2,신사역성일빌딩,,37.514132,127.020563,
3,국기원사거리,,37.499517,127.031495,
4,스탈릿대치R,,37.494668,127.062583,
